## Requirements check

In [ ]:
# Import the required Python packages
import astropy
import numpy
import matplotlib
import PIL
import yaml

If the cell above errors out, refer to `Requirements.md`.

## Read configuration file

In [ ]:
# Read the list of notebooks from a YAML file
with open('../../../config/config.yml', 'r') as f:
    config = yaml.safe_load(f)

# Access the values
data_cube = config['paths']['data_cube']

# Print the values
print(f'Data Cube: {data_cube}')

## Data cube information

This pipeline is meant to be used on 3D data cubes, where the third dimension is the frequency of observation. The two first variables in the next code block represent the frequency of the first observation, and the increase between observations. The next three variables are the cube's dimensions.

If they're not known, they can be found by reading the FITS file's header:

```python
from astropy.io import fits
hdulist = fits.open(data_cube)
hdulist.info()
hdulist[0].header
```

In [ ]:
initial_frequency = 106000
frequency_step = 100

num_layers = 901
image_width = 2048
image_height = 2048

In order to mask the point sources, their positions must be known. This notebook assumes that point source catalogs are available on folder `results`. Otherwise, they can be created by running `pipeline.ipynb`.

The next two variables are the (zero-indexed) positions of the X and Y coordinates of the sources (X_IMAGE and Y_IMAGE) in the parameter list of the catalog. For example, if `default.param` is:

```
NUMBER
X_IMAGE
Y_IMAGE
X_WORLD
Y_WORLD
```

`pos_x_image` should be set to 1, and `pos_y_image` to 2.

In [ ]:
pos_x_image = 1
pos_y_image = 2

We blank out sources by covering them with a circle, whose radius is fixed by the parameter below.

In [ ]:
galaxy_radius = 8 

After all the above variables can be set, the notebook can be ran unattended.

## Mask

In [ ]:
from astropy.io import fits

hdulist = fits.open(data_cube)

header = hdulist[0].header
data = hdulist[0].data

In order to create the mask, we need to "blank out" the sources, setting them to the minimum value found across the cube. We first need to find this minimum value.

In [ ]:
import numpy as np

minimum = np.min(data)
minimum

In [ ]:
import math

distance = lambda x1, x2, y1, y2: math.sqrt( (x2**2-x1**2) + (y2**2-y1**2))

coordinate_pairs = [ (x, y) for x in range(-galaxy_radius, galaxy_radius+1)
                     for y in range(-galaxy_radius, galaxy_radius+1)
                     if distance(0, x, 0, y) <= galaxy_radius]

def mask_galaxy(datos, x, y, layer):
    for (x_offset, y_offset) in coordinate_pairs:
        if 0 <= x+x_offset <= 2047 and 0 <= y+y_offset <= 2047:
            data[layer, y+y_offset, x+x_offset] = minimum

In [ ]:
import os

for layer in range(num_layers):
    frequency = initial_frequency + frequency_step*layer
    catalog = os.path.join("results", f"results{frequency}kHz.cat")
    with open(catalog) as f:        
        for line in f:
            if line[0] == "#":
                continue
            (x, y) = (line.split()[pos_x_image], line.split()[pos_y_image])
            mask_galaxy(data, round(float(x)), round(float(y)), layer)

In [ ]:
hdu = fits.PrimaryHDU(data, header=header)
new_hdulist = fits.HDUList([hdu])
new_hdulist.writeto("mask.fits")
new_hdulist.close()